<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


## OpenAI Assistant with Code Interpreter

Let's work through another example of creating an assistant with the code-interpreter tool!

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

### STEP 1: ASSISTANT CREATION

In [40]:
assistant = client.beta.assistants.create(
    name="Mortgage Bot",
    instructions="You use Python code to help answer questions about mortgage and interest payments.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo"
)

### STEP 2: THREAD CREATION

In [41]:
thread = client.beta.threads.create()

### STEP 3: ADD MESSAGE TO THREAD

In [42]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to buy a house that costs $2.1 Milion on a 30-yr fixed loan at 7.8% interest. What will my monthly payments be?"
)

### STEP 4: RUN THREAD WITH ASSISTANT

In [43]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Give detailed analysis and considerations for the user question."
)

In [44]:
import time

while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
    
    print(run.status)
    
    time.sleep(5)

STILL IN PROGRESS
in_progress
STILL IN PROGRESS
completed


### STEP 5: DISPLAY RESPONSE

In [45]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [46]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_F8cG72YzttpcZCTFVlmrSJFo', assistant_id='asst_cv21kOPIOyRgjXVfpay8pFfc', content=[MessageContentText(text=Text(annotations=[], value='The monthly mortgage payments for a house priced at $2.1 million on a 30-year fixed loan with a 7.8% interest rate will be approximately $15,117.28.'), type='text')], created_at=1701914291, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_NkxEVBA7cqpCBxxUXFqNZUfU', thread_id='thread_amw2BVxDdHbdt0otmq2KfdMJ'), ThreadMessage(id='msg_i5RM3b4yV1Hqkw0iXXkxnKo8', assistant_id='asst_cv21kOPIOyRgjXVfpay8pFfc', content=[MessageContentText(text=Text(annotations=[], value="To calculate your monthly mortgage payments, you can use the formula for calculating the fixed monthly payment on a loan. The formula is:\n\n\\[ M = \\frac{P \\cdot r \\cdot (1+r)^n}{(1+r)^n-1} \\]\n\nWhere:\n- M is the monthly mortgage payment\n- P is the loan amount\n- r is the monthly interest rate (annu

In [47]:
for thread_message in messages.data[::-1]:
    print(thread_message.content[0].text.value)
    print('\n')

I want to buy a house that costs $2.1 Milion on a 30-yr fixed loan at 7.8% interest. What will my monthly payments be?


To calculate your monthly mortgage payments, you can use the formula for calculating the fixed monthly payment on a loan. The formula is:

\[ M = \frac{P \cdot r \cdot (1+r)^n}{(1+r)^n-1} \]

Where:
- M is the monthly mortgage payment
- P is the loan amount
- r is the monthly interest rate (annual interest rate divided by 12)
- n is the total number of payments (number of years multiplied by 12)

Let's calculate the monthly payments for your house.


The monthly mortgage payments for a house priced at $2.1 million on a 30-year fixed loan with a 7.8% interest rate will be approximately $15,117.28.




In [57]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

### STEP 6: CONTINUE CONVERSATION

In [48]:
def wait_on_run(run, thread):
    '''
    Simple function that waits on the run of a particular thread.
    Returns the run once the thread has been run.
    '''
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [49]:
client.beta.threads.messages.list(thread_id=thread.id)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_F8cG72YzttpcZCTFVlmrSJFo', assistant_id='asst_cv21kOPIOyRgjXVfpay8pFfc', content=[MessageContentText(text=Text(annotations=[], value='The monthly mortgage payments for a house priced at $2.1 million on a 30-year fixed loan with a 7.8% interest rate will be approximately $15,117.28.'), type='text')], created_at=1701914291, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_NkxEVBA7cqpCBxxUXFqNZUfU', thread_id='thread_amw2BVxDdHbdt0otmq2KfdMJ'), ThreadMessage(id='msg_i5RM3b4yV1Hqkw0iXXkxnKo8', assistant_id='asst_cv21kOPIOyRgjXVfpay8pFfc', content=[MessageContentText(text=Text(annotations=[], value="To calculate your monthly mortgage payments, you can use the formula for calculating the fixed monthly payment on a loan. The formula is:\n\n\\[ M = \\frac{P \\cdot r \\cdot (1+r)^n}{(1+r)^n-1} \\]\n\nWhere:\n- M is the monthly mortgage payment\n- P is the loan amount\n- r is the monthly interest rate (annu

In [51]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="What if I put a down payment of $200k on the house, how would that change the monthly payments?"
)

In [54]:
# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
# WE CAN REQUEST IN 'ASC' ORDER
# WE CAN ALSO REQUEST ONLY MESSAGES AFTER A PARTICULAR ID!
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)

display_thread_messages(messages)

With a down payment of $200,000 on the house, the new loan amount would be $1,900,000. The new monthly mortgage payment would be approximately $13,677.54.


If you make a down payment of $200,000 on the house, it will decrease the loan amount. To calculate the new monthly mortgage payments, we need to subtract the down payment from the original loan amount. Let's calculate the new monthly payments.




-----

### Optional: Review the Run Steps the program took to get the reply

In [58]:
client.beta.threads.runs.list(thread.id)

SyncCursorPage[Run](data=[Run(id='run_sWXbCHS9kbNmerxQ7oG1jtjF', assistant_id='asst_cv21kOPIOyRgjXVfpay8pFfc', cancelled_at=None, completed_at=1701914636, created_at=1701914622, expires_at=None, failed_at=None, file_ids=[], instructions='You use Python code to help answer questions about mortgage and interest payments.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1701914622, status='completed', thread_id='thread_amw2BVxDdHbdt0otmq2KfdMJ', tools=[ToolAssistantToolsCode(type='code_interpreter')]), Run(id='run_NkxEVBA7cqpCBxxUXFqNZUfU', assistant_id='asst_cv21kOPIOyRgjXVfpay8pFfc', cancelled_at=None, completed_at=1701914292, created_at=1701914278, expires_at=None, failed_at=None, file_ids=[], instructions='Give detailed analysis and considerations for the user question.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1701914278, status='completed', thread_id='thread

In [59]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [60]:
for step in run_steps:
    print(step.step_details)
    print('\n')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_s1HJdFmDlVdEYM05JEQP5oye'), type='message_creation')


ToolCallsStepDetails(tool_calls=[CodeToolCall(id='call_tfnMmvWr2iJOEEH9MG1ItBzO', code_interpreter=CodeInterpreter(input='down_payment = 200000\nnew_loan_amount = loan_amount - down_payment\n\nnew_monthly_payment = (new_loan_amount * monthly_interest_rate * (1 + monthly_interest_rate)**total_payments) / ((1 + monthly_interest_rate)**total_payments - 1)\nnew_monthly_payment', outputs=[CodeInterpreterOutputLogs(logs='13677.539515968123', type='logs')]), type='code_interpreter')], type='tool_calls')


MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_cPivY7mHx7rMtTHWuMEBG9eP'), type='message_creation')




### Optional: Delete Assistant

In [61]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_cv21kOPIOyRgjXVfpay8pFfc', created_at=1701914273, description=None, file_ids=[], instructions='You use Python code to help answer questions about mortgage and interest payments.', metadata={}, model='gpt-3.5-turbo', name='Mortgage Bot', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])]


In [63]:
my_assistants.data[0].id

'asst_cv21kOPIOyRgjXVfpay8pFfc'

In [64]:
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_cv21kOPIOyRgjXVfpay8pFfc', deleted=True, object='assistant.deleted')
